In [13]:
#En esta parte del codigo se instala todas las librerias que van a ser requeridas en este proyecto

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import datetime


ModuleNotFoundError: No module named 'oandapyV20'

In [3]:

## Se definen todas nuestros parametros a utilizar 
def get_prices(date):
    A1_OA_Da = 16                     # Day Align
    A1_OA_Ta = "America/Mexico_City"  # Time Align
    A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
    A1_OA_At = "practice"             # Tipo de cuenta
    A1_OA_In = "USD_MXN"              # Instrumento
    A1_OA_Gn = "M1"                   # Granularidad de velas
    A1_OA_Ak = '2' + '6b62ddbe404c61a1cc9da0ed8395945-52d44ef76c42b62460581783bba6c8e' + 'a'
    ## se obtiene el spot
    a=datetime.datetime.strptime(date,"%m/%d/%Y %H:%M")
    today = a
    DD = datetime.timedelta(minutes=11)
    earlier = today - DD
    later = today + DD
    F1=later.strftime("%Y-%m-%dT%H:%M:%SZ")
    F2=earlier.strftime("%Y-%m-%dT%H:%M:%SZ")
    api = API(access_token=A1_OA_Ak)
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F2, "to": F1}
    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    a=list(map(lambda i:[A1_Hist['candles'][i]['time'],float(A1_Hist['candles'][i]['mid']['o']),float(A1_Hist['candles'][i]['mid']['h']),
                          float(A1_Hist['candles'][i]['mid']['l']),float(A1_Hist['candles'][i]['mid']['c'])],range(len(A1_Hist['candles']))))
    pd_hist = pd.DataFrame(a,columns=['TimeStamp','Open','High','Low','Close'])
    pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    
    return pd_hist 

In [5]:
##Se lee la información historica y se definen los diferentes escenarios
data=pd.read_csv('data.csv')


a=data.loc[[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus>=data.Previous)]]
b=data.loc[[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus<data.Previous)]]
c=data.loc[[a and b for a,b in zip(data.Actual<data.Consensus,data.Consensus>=data.Previous)]]
d=data.loc[[a and b for a,b in zip(data.Actual<=data.Consensus,data.Consensus<data.Previous)]]
data['a']=[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus>=data.Previous)]
data['b']=[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus<data.Previous)]
data['c']=[a and b for a,b in zip(data.Actual<data.Consensus,data.Consensus>=data.Previous)]
data['d']=[a and b for a,b in zip(data.Actual<=data.Consensus,data.Consensus<data.Previous)]

FileNotFoundError: File b'data.csv' does not exist

In [6]:
## Se seleccionan las fechas
week_days=list(map(lambda x:datetime.datetime.weekday(datetime.datetime.strptime(data.DateTime[x],"%m/%d/%Y %H:%M")),range(len(data))))

NameError: name 'data' is not defined

In [ ]:
temp=list(map(lambda x:get_prices(data.DateTime[x]),range(len(data))))

In [ ]:
## Se definen las medidas de desempeño
c1_direction=list(map(lambda x:temp[x].loc[21].Close-temp[x].loc[11].Open,range(len(temp))))

In [ ]:
c2_vol=list(map(lambda x:max(temp[x].High)-min(temp[x].Low),range(len(temp))))

In [ ]:
c3_drawDown=list(map(lambda x:temp[x].loc[0].Open-temp[x].loc[21].Low,range(len(temp))))

In [7]:
c4_drawUp=list(map(lambda x:temp[x].loc[0].Open-temp[x].loc[21].High,range(len(temp))))

NameError: name 'temp' is not defined

In [8]:
## se nombran dentro del Df data
data['c1_direction']=c1_direction
data['c2_vol']=c2_vol
data['c3_drawDown']=c3_drawDown
data['c4_drawUp']=c4_drawUp


NameError: name 'c1_direction' is not defined

In [14]:
## Se calcula su probabilidad de ocurrencia
po=[sum(data['a'])/len(data),
sum(data['b'])/len(data),
sum(data['c'])/len(data),
sum(data['d'])/len(data)]

NameError: name 'data' is not defined

In [15]:
#Se definen las direcciones positivas de los escenarios
tp=np.array([sum(data.loc[a.index]['c1_direction']>0)/sum(data['a']),
sum(data.loc[b.index]['c1_direction']>0)/sum(data['b']),
sum(data.loc[c.index]['c1_direction']>0)/sum(data['c']),
sum(data.loc[d.index]['c1_direction']>0)/sum(data['d'])])

NameError: name 'data' is not defined

In [ ]:
## su complemento
tn=1-tp


In [9]:
results=np.transpose(pd.DataFrame([tp,tn,po],columns=['a','b','c','d'],index=['direccion positiva','direccion negativa','porcentaje ocurrencia']))

NameError: name 'tp' is not defined

In [10]:
## Se definen los parametros de la cuenta y su criterio de apertura de posición que
total=100000
trade=[]
for i in range(len(temp)):
    if (results.loc[data.iloc[i,7:11]]['direccion positiva']==1)[0]:

        dolares_totales=np.trunc(total/temp[i].loc[0].Open)
        monto_compra=dolares_totales*temp[i].loc[11].Open
        total-=monto_compra
        monto_venta=dolares_totales*temp[i].loc[21].Close
        pL=monto_venta-monto_compra
        total+=monto_venta
        trade.append([dolares_totales,pL,total])
    else:
        trade.append([0,0,total])


NameError: name 'temp' is not defined

In [11]:
## Se genera Df de los trades y sus descripciones
trades=pd.DataFrame(trade,columns=['total quantity','P&L','Total'])
trades

,total quantity,P&L,Total


In [12]:
# Se unene las tablas
data=data.join(trades)

NameError: name 'data' is not defined